In [2]:
#dpendencies
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [3]:
#load demographic data and basic transoformation
dem_data = pd.read_csv("data/dem_data.csv")
dem_data = dem_data.rename(columns={"Location":"state"})
dem_data = dem_data.dropna()
dem_data.head(3)

,state,Population Estimate (as of July 1) - 2018 - Both Sexes; Total,Population Estimate (as of July 1) - 2018 - Male; Total,Population Estimate (as of July 1) - 2018 - Female; Total,Population Estimate (as of July 1) - 2018 - Both Sexes; Total - Under 5 years,Population Estimate (as of July 1) - 2018 - Male; Total - Under 5 years,Population Estimate (as of July 1) - 2018 - Female; Total - Under 5 years,Population Estimate (as of July 1) - 2018 - Both Sexes; Total - 5 to 9 years,Population Estimate (as of July 1) - 2018 - Male; Total - 5 to 9 years,Population Estimate (as of July 1) - 2018 - Female; Total - 5 to 9 years,...,Population Estimate (as of July 1) - 2018 - Female; Total - 75 to 79 years,Population Estimate (as of July 1) - 2018 - Both Sexes; Total - 80 to 84 years,Population Estimate (as of July 1) - 2018 - Male; Total - 80 to 84 years,Population Estimate (as of July 1) - 2018 - Female; Total - 80 to 84 years,Population Estimate (as of July 1) - 2018 - Both Sexes; Total - 85 years and over,Population Estimate (as of July 1) - 2018 - Male; Total - 85 years and over,Population Estimate (as of July 1) - 2018 - Female; Total - 85 years and over,Population Estimate (as of July 1) - 2018 - Both Sexes; Median age (years),Population Estimate (as of July 1) - 2018 - Male; Median age (years),Population Estimate (as of July 1) - 2018 - Female; Median age (years)
0,United States,327167434,161128679,166038755,19810275,10132202,9678073,20195642,10315990,9879652,...,5122392,6127308,2590366,3536942,6544503,2325693,4218810,38.2,36.9,39.5
1,Alabama,4887871,2364115,2523756,293203,149264,143939,297900,152067,145833,...,84943,98450,40077,58373,89865,30133,59732,39.2,37.8,40.7
2,Alaska,737438,384573,352865,53115,27082,26033,52056,26798,25258,...,6960,7710,3521,4189,6853,2594,4259,34.6,34.1,35.1


In [66]:
#load COVID data and basic transoformation
covid_data = pd.read_csv("data/us-states.csv")
covid_data = covid_data.dropna()
covid_data.head(3)

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0


In [67]:
#Combine dfs
df_scrap = covid_data.merge(dem_data,on="state")
df_scrap['date'] = pd.to_datetime(df_scrap['date'])

In [68]:
#Create Difference in day by day cases column
df_scrap['diff_cases'] = df_scrap['cases'] - df_scrap['cases'].shift(1)
df_scrap.loc[df_scrap.groupby('state')['diff_cases'].head(1).index, 'diff_cases'] = df_scrap['cases']

In [69]:
#difference in cases 3 days ago
df_scrap['diff_cases3'] = df_scrap['cases'] - df_scrap['cases'].shift(3)
df_scrap.loc[df_scrap.groupby('state')['diff_cases3'].head(3).index, 'diff_cases3'] = df_scrap['cases']

In [70]:
#Create Difference in day by day deaths column
df_scrap['diff_deaths'] = df_scrap['deaths'] - df_scrap['deaths'].shift(1)
df_scrap.loc[df_scrap.groupby('state')['diff_deaths'].head(1).index, 'diff_deaths'] = df_scrap['deaths']

In [71]:
#differenc in deaths 3 days ago 
df_scrap['diff_deaths3'] = df_scrap['deaths'] - df_scrap['deaths'].shift(3)
df_scrap.loc[df_scrap.groupby('state')['diff_deaths3'].head(3).index, 'diff_deaths3'] = df_scrap['deaths']


In [72]:
#extra data I am toying around with
df_latest = df_scrap.groupby('state').last()
df_earliest = df_scrap.groupby('state').first()
pandemic_length = df_latest['date']-df_earliest['date']
df_latest['length'] = pandemic_length
df_latest.head(3)

,date,fips,cases,deaths,Population Estimate (as of July 1) - 2018 - Both Sexes; Total,Population Estimate (as of July 1) - 2018 - Male; Total,Population Estimate (as of July 1) - 2018 - Female; Total,Population Estimate (as of July 1) - 2018 - Both Sexes; Total - Under 5 years,Population Estimate (as of July 1) - 2018 - Male; Total - Under 5 years,Population Estimate (as of July 1) - 2018 - Female; Total - Under 5 years,...,Population Estimate (as of July 1) - 2018 - Male; Total - 85 years and over,Population Estimate (as of July 1) - 2018 - Female; Total - 85 years and over,Population Estimate (as of July 1) - 2018 - Both Sexes; Median age (years),Population Estimate (as of July 1) - 2018 - Male; Median age (years),Population Estimate (as of July 1) - 2018 - Female; Median age (years),diff_cases,diff_cases3,diff_deaths,diff_deaths3,length
state,,,,,,,,,,,,,,,,,,,,,
Alabama,2020-05-13,1,10700,450,4887871,2364115,2523756,293203,149264,143939,...,30133,59732,39.2,37.8,40.7,236.0,811.0,15.0,57.0,61 days
Alaska,2020-05-13,2,383,10,737438,384573,352865,53115,27082,26033,...,2594,4259,34.6,34.1,35.1,0.0,5.0,2.0,2.0,62 days
Arizona,2020-05-13,4,12176,594,7171646,3565339,3606307,435936,222960,212976,...,58181,84094,37.9,36.6,39.3,440.0,1057.0,32.0,58.0,108 days


In [73]:
df_scrap = pd.get_dummies(df_scrap, columns = ['date'])
df_scrap = df_scrap.groupby('state')

In [76]:
#head of the grouped data (What Vikash was asking for)
df_scrap.head(2)

,state,fips,cases,deaths,Population Estimate (as of July 1) - 2018 - Both Sexes; Total,Population Estimate (as of July 1) - 2018 - Male; Total,Population Estimate (as of July 1) - 2018 - Female; Total,Population Estimate (as of July 1) - 2018 - Both Sexes; Total - Under 5 years,Population Estimate (as of July 1) - 2018 - Male; Total - Under 5 years,Population Estimate (as of July 1) - 2018 - Female; Total - Under 5 years,...,date_2020-05-04 00:00:00,date_2020-05-05 00:00:00,date_2020-05-06 00:00:00,date_2020-05-07 00:00:00,date_2020-05-08 00:00:00,date_2020-05-09 00:00:00,date_2020-05-10 00:00:00,date_2020-05-11 00:00:00,date_2020-05-12 00:00:00,date_2020-05-13 00:00:00
0,Washington,53,1,0,7535591,3769911,3765680,462701,237614,225087,...,0,0,0,0,0,0,0,0,0,0
1,Washington,53,1,0,7535591,3769911,3765680,462701,237614,225087,...,0,0,0,0,0,0,0,0,0,0
114,Illinois,17,1,0,12741080,6260901,6480179,760619,389047,371572,...,0,0,0,0,0,0,0,0,0,0
115,Illinois,17,1,0,12741080,6260901,6480179,760619,389047,371572,...,0,0,0,0,0,0,0,0,0,0
225,California,6,1,0,39557045,19663577,19893468,2441300,1249515,1191785,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3625,Montana,30,6,0,1062305,534852,527453,62536,31959,30577,...,0,0,0,0,0,0,0,0,0,0
3686,Puerto Rico,72,3,0,3195153,1518043,1677110,126621,64719,61902,...,0,0,0,0,0,0,0,0,0,0
3687,Puerto Rico,72,4,0,3195153,1518043,1677110,126621,64719,61902,...,0,0,0,0,0,0,0,0,0,0
3748,West Virginia,54,1,0,1805832,894340,911492,95257,48695,46562,...,0,0,0,0,0,0,0,0,0,0


In [12]:
######################################
###Model I'm working on from here down feel free to work on or do your own
######################################
target_column = ['diff_deaths3']
predictors = list(set(list(df1.columns))-set(target_column))
df1[predictors] = df1[predictors]/df1[predictors].max()
df1.describe()


,fips,cases,deaths,Population Estimate (as of July 1) - 2018 - Both Sexes; Total,Population Estimate (as of July 1) - 2018 - Male; Total,Population Estimate (as of July 1) - 2018 - Female; Total,Population Estimate (as of July 1) - 2018 - Both Sexes; Total - Under 5 years,Population Estimate (as of July 1) - 2018 - Male; Total - Under 5 years,Population Estimate (as of July 1) - 2018 - Female; Total - Under 5 years,Population Estimate (as of July 1) - 2018 - Both Sexes; Total - 5 to 9 years,...,date_2020-05-04,date_2020-05-05,date_2020-05-06,date_2020-05-07,date_2020-05-08,date_2020-05-09,date_2020-05-10,date_2020-05-11,date_2020-05-12,date_2020-05-13
count,3806.000000,3806.000000,3806.000000,3806.000000,3806.000000,3806.000000,3806.000000,3806.000000,3806.000000,3806.000000,...,3806.000000,3806.000000,3806.000000,3806.000000,3806.000000,3806.000000,3806.000000,3806.000000,3806.000000,3806.000000
mean,0.412850,0.028143,0.018946,0.177908,0.176378,0.179421,0.174723,0.174613,0.174838,0.174933,...,0.013663,0.013663,0.013663,0.013663,0.013663,0.013663,0.013663,0.013663,0.013663,0.013663
std,0.232281,0.089124,0.079967,0.202553,0.201679,0.203445,0.206460,0.206328,0.206598,0.205364,...,0.116101,0.116101,0.116101,0.116101,0.116101,0.116101,0.116101,0.116101,0.116101,0.116101
min,0.013889,0.000003,0.000000,0.014605,0.014979,0.014236,0.012158,0.012226,0.012087,0.012565,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.222222,0.000252,0.000036,0.048772,0.048993,0.048553,0.050893,0.050867,0.050921,0.053172,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.402778,0.003500,0.001056,0.123565,0.120228,0.126864,0.120101,0.119458,0.120776,0.121165,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.611111,0.018657,0.008160,0.190499,0.191721,0.189292,0.189531,0.190165,0.188865,0.187599,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
X = df1[predictors].values
y = df1[target_column].values


KeyError: "None of [Int64Index([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,\n            ...\n            50, 50, 51, 51, 52, 53, 54, 57, 58, 59],\n           dtype='int64', length=3806)] are in the [columns]"

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

dtree = DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)

dtree.fit(X_train, y_train)

In [ ]:
#RF model
model_rf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=100)
model_rf.fit(X_train, y_train) 
pred_train_rf= model_rf.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_rf)))
print(r2_score(y_train, pred_train_rf))

pred_test_rf = model_rf.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_rf)))
print(r2_score(y_test, pred_test_rf))